# LOADING DATASET:

In [1]:
import pandas as pd
ds = pd.read_excel('Bank_Personal_Loan_Modelling.xlsx' , sheet_name=1)
ds.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [11]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


# CHEKING NULL VALUES:

In [4]:
ds.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [5]:
ds.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

# DROPPING UNNECESSARY FEATURES

In [2]:
ds.drop(['ID','ZIP Code'],axis=1, inplace=True)
ds.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


# CONVERTING FLOAT TO INTEGER:

In [3]:
import numpy as np
ds['CCAvg'] = np.round(ds['CCAvg'])
ds.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,2.0,1,0,0,1,0,0,0
1,45,19,34,3,2.0,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,3.0,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


# APPLYING RANDOM FOREST

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=1000 , max_features=2, oob_score=True)

features = ['Age', 'Experience', 'Income','Family', 'CCAvg','Education', 'Mortgage','Securities Account',
       'CD Account', 'Online', 'CreditCard']

rf_model.fit(X=ds[features] , y=ds['Personal Loan'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

# CHEKING MODEL ACCURACY:

In [5]:
print(rf_model.oob_score_)

0.9864


In [6]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

Age 0.049941899159073905
Experience 0.049671200552768906
Income 0.36219480502807516
Family 0.10406462359967314
CCAvg 0.1403450989071427
Education 0.16640855641140678
Mortgage 0.04669172538233816
Securities Account 0.006333089688344736
CD Account 0.05399865642093584
Online 0.009581521395744386
CreditCard 0.010768823454496299


# MOST IMPORTANT FEATURES:
1]INCOME - 0.36

2]CCAVG - 0.14

3]EDUCATION - 0.16

# APPLYING DECISION TREE:

In [7]:
from sklearn import tree
dt_model = tree.DecisionTreeClassifier(max_depth=6,max_leaf_nodes=10)
predictors = pd.DataFrame([ds['Income'],ds['CCAvg'],ds['Education']]).T

dt_model.fit(X=predictors , y=ds['Personal Loan'])

with open('bankloan.dot','w') as f:
    f = tree.export_graphviz(dt_model , feature_names=['Income','CCAvg','Education'],out_file=f)



In [8]:
dt_model.score(X=predictors , y=ds['Personal Loan'])

0.9708